In [3]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/SLT final project/2 Hours- 4 Countries English Speech Synthesis Corpus/2 Hours- 4 Countries English Speech Synthesis Corpus/data/New_zealand_data/Wave'

Mounted at /content/drive


In [ ]:
!pip install pip==23.3.1 #needs pip < 24.1 for whatever
!pip install fairseq
!git clone https://github.com/facebookresearch/fairseq.git

In [5]:
!pip install pydub
!apt-get install ffmpeg

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [15]:
#down load vocoder
!wget https://dl.fbaipublicfiles.com/fairseq/speech_to_speech/vocoder/code_hifigan/hubert_base_100_lj/g_00500000
!wget https://dl.fbaipublicfiles.com/fairseq/speech_to_speech/vocoder/code_hifigan/hubert_base_100_lj/config.json

--2024-12-11 04:19:26--  https://dl.fbaipublicfiles.com/fairseq/speech_to_speech/vocoder/code_hifigan/hubert_base_100_lj/g_00500000
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.244.202.103, 18.244.202.62, 18.244.202.25, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.244.202.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53590413 (51M) [binary/octet-stream]
Saving to: ‘g_00500000.1’

g_00500000.1        100%[===================>]  51.11M  34.6MB/s    in 1.5s    

2024-12-11 04:19:28 (34.6 MB/s) - ‘g_00500000.1’ saved [53590413/53590413]

--2024-12-11 04:19:28--  https://dl.fbaipublicfiles.com/fairseq/speech_to_speech/vocoder/code_hifigan/hubert_base_100_lj/config.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.244.202.103, 18.244.202.62, 18.244.202.25, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.244.202.103|:443... connected.
HTTP request sent, awaiting response... 200 O

In [55]:
#download hubert base + quantizer
!wget https://dl.fbaipublicfiles.com/hubert/hubert_base_ls960.pt
!wget https://dl.fbaipublicfiles.com/textless_nlp/gslm/hubert/km100/km.bin

--2024-12-11 06:07:41--  https://dl.fbaipublicfiles.com/hubert/hubert_base_ls960.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.244.202.103, 18.244.202.73, 18.244.202.62, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.244.202.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1136468879 (1.1G) [application/zip]
Saving to: ‘hubert_base_ls960.pt.2’

hubert_base_ls960.p 100%[===================>]   1.06G   274MB/s    in 4.4s    

2024-12-11 06:07:46 (248 MB/s) - ‘hubert_base_ls960.pt.2’ saved [1136468879/1136468879]

--2024-12-11 06:07:46--  https://dl.fbaipublicfiles.com/textless_nlp/gslm/hubert/km100/km.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.244.202.103, 18.244.202.73, 18.244.202.62, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.244.202.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308469 (301K) [application/octet-stream]
Saving to: ‘km

In [6]:
# Modify transcriptions.txt to be filtered
import re

def clean_text_file(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            # Split the line into ID and text/transcription parts
            parts = line.strip().split('\t', maxsplit=1)
            if len(parts) == 2:
                identifier, text = parts
                # Remove unwanted characters from the text
                cleaned_text = re.sub(r'[\-/%\.]', '', text)  # Remove '-', '/', '%', '.'
                cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Normalize spaces
                # Append .wav to the identifier
                identifier_wav = f"{identifier}.wav"
                # Write to output file in the desired format
                outfile.write(f"{identifier_wav}\t{cleaned_text}\n")

# Input and output file names
input_filename =  "/content/drive/MyDrive/SLT final project/2 Hours- 4 Countries English Speech Synthesis Corpus/2 Hours- 4 Countries English Speech Synthesis Corpus/data/New_zealand_data/ProsodyLabeling/text.txt"
output_filename = "/content/drive/MyDrive/SLT final project/2 Hours- 4 Countries English Speech Synthesis Corpus/2 Hours- 4 Countries English Speech Synthesis Corpus/data/New_zealand_data/ProsodyLabeling/filtered_transcriptions"

# Run the function
clean_text_file(input_filename, output_filename)


In [7]:
import os
from pydub import AudioSegment

# Function to convert audio
def convert_audio(input_file, output_file, target_sample_rate=16000):
    # Load the .wav file
    audio = AudioSegment.from_wav(input_file)

    # Set the target sample rate and export
    audio = audio.set_frame_rate(target_sample_rate)
    audio.export(output_file, format="wav")

# Directory with .wav files
input_dir = data_dir
output_dir = '/content/drive/MyDrive/SLT final project/2 Hours- 4 Countries English Speech Synthesis Corpus/2 Hours- 4 Countries English Speech Synthesis Corpus/data/New_zealand_data/16hz_wave'

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop through all files in the directory and convert
for filename in os.listdir(input_dir):
    if filename.endswith('.wav'):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        # Convert to 16 kHz
        convert_audio(input_path, output_path)
        print(f"Converted {filename} to 16 kHz")


Converted 000003.wav to 16 kHz
Converted 000002.wav to 16 kHz
Converted 000001.wav to 16 kHz
Converted 000004.wav to 16 kHz
Converted 000007.wav to 16 kHz
Converted 000019.wav to 16 kHz
Converted 000011.wav to 16 kHz
Converted 000014.wav to 16 kHz
Converted 000015.wav to 16 kHz
Converted 000013.wav to 16 kHz
Converted 000012.wav to 16 kHz
Converted 000006.wav to 16 kHz
Converted 000016.wav to 16 kHz
Converted 000020.wav to 16 kHz
Converted 000008.wav to 16 kHz
Converted 000010.wav to 16 kHz
Converted 000009.wav to 16 kHz
Converted 000018.wav to 16 kHz
Converted 000005.wav to 16 kHz
Converted 000017.wav to 16 kHz
Converted 000028.wav to 16 kHz
Converted 000033.wav to 16 kHz
Converted 000035.wav to 16 kHz
Converted 000031.wav to 16 kHz
Converted 000029.wav to 16 kHz
Converted 000025.wav to 16 kHz
Converted 000032.wav to 16 kHz
Converted 000030.wav to 16 kHz
Converted 000027.wav to 16 kHz
Converted 000036.wav to 16 kHz
Converted 000024.wav to 16 kHz
Converted 000021.wav to 16 kHz
Converte

In [33]:
import os
import random
from shutil import copyfile

# Change to the working directory
%cd /content

def split_data(data_dir, output_dir, train_ratio=0.8, test_ratio=0.1):
    """
    Splits the data directory into train, validation, and test sets.

    Args:
        data_dir (str): Path to the original data directory (contains .wav and transcriptions.txt).
        output_dir (str): Path to the output directory.
        train_ratio (float): Proportion of data to use for training (default 0.8).
        test_ratio (float): Proportion of data to use for testing (default 0.1).
    """
    # Ensure train_ratio + test_ratio <= 1
    if train_ratio + test_ratio > 1:
        raise ValueError("Train and test ratios must sum to at most 1.")

    # Path to the transcription file
    transcription_file = os.path.join(
        "/content/drive/MyDrive/SLT final project/2 Hours- 4 Countries English Speech Synthesis Corpus/2 Hours- 4 Countries English Speech Synthesis Corpus/data/New_zealand_data/ProsodyLabeling/filtered_transcriptions"
    )

    # Read transcriptions
    with open(transcription_file, "r") as f:
        lines = f.readlines()

    # Shuffle the data
    random.shuffle(lines)

    # Calculate split indices
    total_lines = len(lines)
    train_idx = int(total_lines * train_ratio)
    test_idx = int(total_lines * test_ratio) + train_idx

    # Split the dataset
    train_lines = lines[:train_idx]
    test_lines = lines[train_idx:test_idx]
    valid_lines = lines[test_idx:]

    # Create output directories
    train_dir = os.path.join(output_dir, "train")
    valid_dir = os.path.join(output_dir, "valid")
    test_dir = os.path.join(output_dir, "test")
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(valid_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Write split transcriptions
    with open(os.path.join(train_dir, "transcriptions.txt"), "w") as f:
        f.writelines(train_lines)

    with open(os.path.join(valid_dir, "transcriptions.txt"), "w") as f:
        f.writelines(valid_lines)

    with open(os.path.join(test_dir, "transcriptions.txt"), "w") as f:
        f.writelines(test_lines)

    # Copy corresponding audio files
    def copy_files(lines, destination_dir):
        for line in lines:
            audio_file = line.split("\t")[0].strip()
            source_path = os.path.join(data_dir, audio_file)
            dest_path = os.path.join(destination_dir, audio_file)
            if os.path.exists(source_path):
                copyfile(source_path, dest_path)

    copy_files(train_lines, train_dir)
    copy_files(valid_lines, valid_dir)
    copy_files(test_lines, test_dir)

    print(f"Data successfully split into train ({train_dir}), valid ({valid_dir}), and test ({test_dir})")

# Path to the output directory
output_dir = "processed_data"

data_dir_16hz = '/content/drive/MyDrive/SLT final project/2 Hours- 4 Countries English Speech Synthesis Corpus/2 Hours- 4 Countries English Speech Synthesis Corpus/data/New_zealand_data/16hz_wave'

# Split the data
split_data(data_dir_16hz, output_dir)


/content
Data successfully split into train (processed_data/train), valid (processed_data/valid), and test (processed_data/test)


In [9]:
!pip install gtts

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [35]:
import os
from gtts import gTTS

# Ensure the folder exists
output_folder = "source_wav"
train_output_folder = "source_wav/train"
test_output_folder = "source_wav/test"
valid_output_folder = "source_wav/valid"
os.makedirs(valid_output_folder, exist_ok=True)
os.makedirs(test_output_folder, exist_ok=True)
os.makedirs(train_output_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

source_transcription_path_train = '/content/processed_data/train/transcriptions.txt'
source_transcription_path_test = '/content/processed_data/test/transcriptions.txt'
source_transcription_path_valid = '/content/processed_data/valid/transcriptions.txt'
# Open the transcription file
with open(source_transcription_path_train, "r") as file:
    for line in file:
        # Split the line into filename and text
        parts = line.strip().split("\t")
        if len(parts) != 2:
            continue  # Skip invalid lines
        filename, text = parts
        # Generate TTS audio
        tts = gTTS(text, lang="en")
        # Save the file
        output_path = os.path.join(output_folder, 'train' ,filename)
        tts.save(output_path)

with open(source_transcription_path_test, "r") as file:
    for line in file:
        # Split the line into filename and text
        parts = line.strip().split("\t")
        if len(parts) != 2:
            continue  # Skip invalid lines
        filename, text = parts
        # Generate TTS audio
        tts = gTTS(text, lang="en")
        # Save the file
        output_path = os.path.join(output_folder, 'test' ,filename)
        tts.save(output_path)

with open(source_transcription_path_valid, "r") as file:
    for line in file:
        # Split the line into filename and text
        parts = line.strip().split("\t")
        if len(parts) != 2:
            continue  # Skip invalid lines
        filename, text = parts
        # Generate TTS audio
        tts = gTTS(text, lang="en")
        # Save the file
        output_path = os.path.join(output_folder, 'valid' ,filename)
        tts.save(output_path)
print(f"Synthetic TTS .wav files saved in '{output_folder}'")


Synthetic TTS .wav files saved in 'source_wav'


In [18]:
#create manifest files
!pip install soundfile

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [36]:
!PYTHONPATH=/content/fairseq python /content/fairseq/examples/wav2vec/wav2vec_manifest.py /content/source_wav/test --dest /content/source_wav/test --ext wav --valid-percent 0
!PYTHONPATH=/content/fairseq python /content/fairseq/examples/wav2vec/wav2vec_manifest.py /content/source_wav/train --dest /content/source_wav/train --ext wav --valid-percent 0
!PYTHONPATH=/content/fairseq python /content/fairseq/examples/wav2vec/wav2vec_manifest.py /content/target_wav/test --dest /content/target_wav/test --ext wav --valid-percent 0
!PYTHONPATH=/content/fairseq python /content/fairseq/examples/wav2vec/wav2vec_manifest.py /content/target_wav/train --dest /content/target_wav/train --ext wav --valid-percent 0

!PYTHONPATH=/content/fairseq python /content/fairseq/examples/wav2vec/wav2vec_manifest.py /content/source_wav/valid --dest /content/source_wav/valid --ext wav --valid-percent 0
!PYTHONPATH=/content/fairseq python /content/fairseq/examples/wav2vec/wav2vec_manifest.py /content/target_wav/valid --dest /content/target_wav/valid --ext wav --valid-percent 0

In [56]:
!MANIFEST=/content/target_wav/train/train.tsv
!OUT_QUANTIZED_FILE=/content/target_wav/train.txt
!KM_MODEL_PATH=/content/km.bin

!PYTHONPATH=/content/fairseq python /content/fairseq/examples/textless_nlp/gslm/speech2unit/clustering/quantize_with_kmeans.py \
    --feature_type hubert \
    --kmeans_model_path /content/km.bin \
    --acoustic_model_path /content/hubert_base_ls960.pt \
    --layer 6 \
    --manifest_path /content/target_wav/train/train.tsv \
    --out_quantized_file_path /content/target_wav/train.txt \
    --extension ".wav"

2024-12-11 06:08:00.588458: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 06:08:00.605936: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 06:08:00.626590: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 06:08:00.632869: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 06:08:00.647685: I tensorflow/core/platform/cpu_feature_guar

In [58]:
!MANIFEST=/content/target_wav/test/test.tsv
!OUT_QUANTIZED_FILE=/content/target_wav/test.txt
!KM_MODEL_PATH=/content/km.bin

!PYTHONPATH=/content/fairseq python /content/fairseq/examples/textless_nlp/gslm/speech2unit/clustering/quantize_with_kmeans.py \
    --feature_type hubert \
    --kmeans_model_path /content/km.bin \
    --acoustic_model_path /content/hubert_base_ls960.pt \
    --layer 6 \
    --manifest_path /content/target_wav/test/train.tsv \
    --out_quantized_file_path /content/target_wav/test.txt \
    --extension ".wav"

2024-12-11 06:08:52.692127: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 06:08:52.710497: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 06:08:52.733826: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 06:08:52.740354: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 06:08:52.756475: I tensorflow/core/platform/cpu_feature_guar

In [59]:
!MANIFEST=/content/target_wav/valid/valid.tsv
!OUT_QUANTIZED_FILE=/content/target_wav/valid.txt
!KM_MODEL_PATH=/content/km.bin

!PYTHONPATH=/content/fairseq python /content/fairseq/examples/textless_nlp/gslm/speech2unit/clustering/quantize_with_kmeans.py \
    --feature_type hubert \
    --kmeans_model_path /content/km.bin \
    --acoustic_model_path /content/hubert_base_ls960.pt \
    --layer 6 \
    --manifest_path /content/target_wav/valid/train.tsv \
    --out_quantized_file_path /content/target_wav/valid.txt \
    --extension ".wav"

2024-12-11 06:09:09.998549: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 06:09:10.015263: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 06:09:10.035951: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 06:09:10.042174: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 06:09:10.056891: I tensorflow/core/platform/cpu_feature_guar

In [60]:
# $SPLIT1, $SPLIT2, etc. are split names such as train, dev, test, etc.
!PYTHONPATH=/content/fairseq python /content/fairseq/examples/speech_to_speech/preprocessing/prep_s2ut_data.py \
  --source-dir /content/source_wav --target-dir /content/target_wav/ --data-split train test valid \
  --output-root /content/data_root --reduce-unit \
  --vocoder-checkpoint /content/g_00500000 --vocoder-cfg /content/config.json

2024-12-11 06:09:27.351089: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 06:09:27.367611: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 06:09:27.387911: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 06:09:27.394071: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 06:09:27.408422: I tensorflow/core/platform/cpu_feature_guar

In [62]:
!fairseq-train /content/data_root \
  --config-yaml /content/data_root/config.yaml  \
  --task speech_to_speech --target-is-code --target-code-size 100 --vocoder code_hifigan  \
  --criterion speech_to_unit --label-smoothing 0.2 \
  --arch s2ut_transformer_fisher --share-decoder-input-output-embed \
  --dropout 0.1 --attention-dropout 0.1 --relu-dropout 0.1 \
  --train-subset train  \
  --save-dir /content/models \
  --lr 0.0005 --lr-scheduler inverse_sqrt --warmup-init-lr 1e-7 --warmup-updates 10000 \
  --optimizer adam --adam-betas "(0.9,0.98)" --clip-norm 10.0 \
  --max-update 400000 --max-tokens 20000 --max-target-positions 3000 --update-freq 1 \
  --seed 1 --fp16 --num-workers 8 --max-epoch 80 --save-interval 10

2024-12-11 06:12:58.163412: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 06:12:58.180804: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 06:12:58.201602: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 06:12:58.207917: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 06:12:58.223349: I tensorflow/core/platform/cpu_feature_guar

In [63]:
!fairseq-generate /content/data_root \
  --config-yaml config.yaml  \
  --task speech_to_speech --target-is-code --target-code-size 100 --vocoder code_hifigan \
  --path /content/models/checkpoint_best.pt  --gen-subset test \
  --max-tokens 50000 \
  --beam 10 --max-len-a 1 \
  --results-path /content/results

2024-12-11 06:17:57.461652: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 06:17:57.478867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 06:17:57.499603: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 06:17:57.505838: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 06:17:57.520541: I tensorflow/core/platform/cpu_feature_guar

In [64]:
!grep "^D\-" /content/results/generate-test.txt | \
  sed 's/^D-//ig' | sort -nk1 | cut -f3 \
  > /content/results/generate-test.unit

!PYTHONPATH=/content/fairseq python /content/fairseq/examples/speech_to_speech/generate_waveform_from_code.py \
  --in-code-file /content/results/generate-test.unit \
  --vocoder /content/g_00500000 --vocoder-cfg /content/config.json  \
  --results-path /content/results --dur-prediction

2024-12-11 06:18:41.035861: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 06:18:41.052813: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 06:18:41.073437: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 06:18:41.079658: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 06:18:41.094273: I tensorflow/core/platform/cpu_feature_guar